In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
from tensorflow import keras
from toai.imports import *

#from data import DataContainer
#from image import *
#from image_learner import ImageLearner
#from transformations import ImageParser, ImageCropTransformation, ImageRandomCropTransformation

/home/martynas/ml/.venv/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
tf.debugging.set_log_device_placement(False)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            print(gpu)
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
1 Physical GPUs, 1 Logical GPUs


In [4]:
__import__('tensorflow').__version__

'2.0.0'

In [5]:
DATA_DIR = Path('data')
TEMP_DIR = Path('temp')
DATA_DIR.mkdir(parents=True, exist_ok=True)
TEMP_DIR.mkdir(parents=True, exist_ok=True)

In [10]:
df = pd.read_csv('train.csv', low_memory=False)

In [13]:
df.head().T

,0,1,2,3,4
category_id,19,19,0,0,0
date_captured,2011-05-13 23:43:18,2012-03-17 03:48:44,2014-05-11 11:56:46,2013-10-06 02:00:00,2011-07-12 13:11:16
file_name,5998cfa4-23d2-11e8-a6a3-ec086b02610b.jpg,588a679f-23d2-11e8-a6a3-ec086b02610b.jpg,59279ce3-23d2-11e8-a6a3-ec086b02610b.jpg,5a2af4ab-23d2-11e8-a6a3-ec086b02610b.jpg,599fbd89-23d2-11e8-a6a3-ec086b02610b.jpg
frame_num,1,2,1,1,3
id,5998cfa4-23d2-11e8-a6a3-ec086b02610b,588a679f-23d2-11e8-a6a3-ec086b02610b,59279ce3-23d2-11e8-a6a3-ec086b02610b,5a2af4ab-23d2-11e8-a6a3-ec086b02610b,599fbd89-23d2-11e8-a6a3-ec086b02610b
location,33,115,96,57,46
rights_holder,Justin Brown,Justin Brown,Erin Boydston,Erin Boydston,Justin Brown
seq_id,6f084ccc-5567-11e8-bc84-dca9047ef277,6f12067d-5567-11e8-b3c0-dca9047ef277,6faa92d1-5567-11e8-b1ae-dca9047ef277,6f7d4702-5567-11e8-9e03-dca9047ef277,6f1728a1-5567-11e8-9be7-dca9047ef277
seq_num_frames,3,3,1,1,3
width,1024,1024,1024,1024,1024


In [14]:
df.describe()

,category_id,frame_num,location,seq_num_frames,width,height
count,196299.000000,196299.000000,196299.00000,196299.000000,196299.0,196299.000000
mean,4.085069,1.404047,65.83003,1.793494,1024.0,747.486600
std,6.749477,0.739637,34.35272,1.026838,0.0,3.128948
min,0.000000,1.000000,0.00000,1.000000,1024.0,747.000000
25%,0.000000,1.000000,30.00000,1.000000,1024.0,747.000000
50%,0.000000,1.000000,70.00000,1.000000,1024.0,747.000000
75%,8.000000,2.000000,96.00000,3.000000,1024.0,747.000000
max,22.000000,5.000000,138.00000,5.000000,1024.0,768.000000


In [15]:
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196299 entries, 0 to 196298
Data columns (total 11 columns):
category_id       196299 non-null int64
date_captured     196299 non-null object
file_name         196299 non-null object
frame_num         196299 non-null int64
id                196299 non-null object
location          196299 non-null int64
rights_holder     196299 non-null object
seq_id            196299 non-null object
seq_num_frames    196299 non-null int64
width             196299 non-null int64
height            196299 non-null int64
dtypes: int64(6), object(5)
memory usage: 16.5+ MB


In [23]:
df['path'] = df['file_name'].apply(lambda fn : DATA_DIR/'train'/fn)

In [24]:
df.head().T

,0,1,2,3,4
category_id,19,19,0,0,0
date_captured,2011-05-13 23:43:18,2012-03-17 03:48:44,2014-05-11 11:56:46,2013-10-06 02:00:00,2011-07-12 13:11:16
file_name,5998cfa4-23d2-11e8-a6a3-ec086b02610b.jpg,588a679f-23d2-11e8-a6a3-ec086b02610b.jpg,59279ce3-23d2-11e8-a6a3-ec086b02610b.jpg,5a2af4ab-23d2-11e8-a6a3-ec086b02610b.jpg,599fbd89-23d2-11e8-a6a3-ec086b02610b.jpg
frame_num,1,2,1,1,3
id,5998cfa4-23d2-11e8-a6a3-ec086b02610b,588a679f-23d2-11e8-a6a3-ec086b02610b,59279ce3-23d2-11e8-a6a3-ec086b02610b,5a2af4ab-23d2-11e8-a6a3-ec086b02610b,599fbd89-23d2-11e8-a6a3-ec086b02610b
location,33,115,96,57,46
rights_holder,Justin Brown,Justin Brown,Erin Boydston,Erin Boydston,Justin Brown
seq_id,6f084ccc-5567-11e8-bc84-dca9047ef277,6f12067d-5567-11e8-b3c0-dca9047ef277,6faa92d1-5567-11e8-b1ae-dca9047ef277,6f7d4702-5567-11e8-9e03-dca9047ef277,6f1728a1-5567-11e8-9be7-dca9047ef277
seq_num_frames,3,3,1,1,3
width,1024,1024,1024,1024,1024
